In [1]:
#importing libraries and initialzing GPU
import os
from pathlib import Path

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from PIL import Image

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.models import resnet18
from torchvision.models import resnet152
import pytorch_lightning as pl
import sys
from tqdm import tqdm
torch.cuda.init()
assert torch.cuda.is_initialized()
print(torch.cuda.get_device_properties(0))



2023-05-01 17:57:03.831103: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 17:57:03.954259: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-01 17:57:04.630777: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-05-01 17:57:04.630845: W tensorflow/

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [2]:
!{sys.executable} -m pip install slideflow[tf] cucim cupy-cuda11x

Looking in indexes: https://devpi.svc.ngsci.local/root/pypi/+simple/
You should consider upgrading via the '/opt/venv/default/bin/python -m pip install --upgrade pip' command.


In [3]:
!{sys.executable} export SF_SLIDE_BACKEND=libvips

/opt/venv/default/bin/python: can't open file 'export': [Errno 2] No such file or directory


In [4]:
BRCA_ROOT = Path().home() / "datasets" / "brca-psj-path"
CONTEST_DIR = BRCA_ROOT / "contest-phase-2"
IMAGE_ROOT = CONTEST_DIR / "png-downsampled-train"
TABLE_DIR = CONTEST_DIR / "csv-train"
LOGGER_DIR = Path().home() / "logs"

In [5]:

# goal: create slide_id : cancer stage mapping
slide_biop_df = pd.read_csv(CONTEST_DIR / "slide-manifest-train.csv")
outcomes_df = pd.read_csv(TABLE_DIR / "outcomes.csv")
slide_stage_df = slide_biop_df.merge(outcomes_df, on="biopsy_id")

# map cancer stage to 0 - 4:
# outcomes_df["stage"].unique()
#     ['IA', 'IIB', 'IIA', '0', nan, 'IIIC', 'IV', 'IIIA', 'IIIB', 'IB']
def stage_to_int(stage):
    if stage == "0":
        return "0"
    elif stage == "IA" or stage == "IB":
        return "1"
    elif stage == "IIA" or stage == "IIB":
        return "2"
    elif stage == "IIIA" or stage == "IIIB" or stage == "IIIC":
        return "3"
    elif stage == "IV":
        return "4"
    else:
        return ""


slide_stage_df["stage"] = slide_stage_df["stage"].apply(stage_to_int)
slide_stage_df["dataset"]="train"
change=slide_stage_df.sample(frac = 0.2).index
slide_stage_df.loc[change,'dataset'] = "eval"

# subset columns, drop nans, reset index
labels_df = (
    slide_stage_df[["slide_id", "biopsy_id", "stage","dataset"]]
    .copy()
    .dropna(how="any")
    .reset_index(drop=True)
)

labels_df["stage"] = labels_df["stage"]
labels_df.rename(columns = {'slide_id':'patient'}, inplace = True)
labels_df.head(5)
labels_df.to_csv('annotations.csv',index=False)

In [6]:
import slideflow as sf
from slideflow.model import build_feature_extractor
from slideflow.model import list_extractors
list_extractors()

2023-05-01 17:57:13.923798: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-01 17:57:13.924577: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-01 17:57:13.925256: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-01 17:57:13.925970: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-01 17:57:13.927091: I tensorflow/compiler/xla/stream_executo

['nasnet_large_imagenet',
 'xception_imagenet',
 'resnet152_imagenet',
 'retccl',
 'mnasnet_imagenet',
 'resnet101_imagenet',
 'mobilenet_imagenet',
 'vgg19_imagenet',
 'mobilenet_v3_large_imagenet',
 'vgg16_imagenet',
 'resnet152_v2_imagenet',
 'densenet_imagenet',
 'mobilenet_v2_imagenet',
 'googlenet_imagenet',
 'inception_imagenet',
 'resnet50_imagenet',
 'shufflenet_imagenet',
 'squeezenet_imagenet',
 'resnet101_v2_imagenet',
 'mobilenet_v3_small_imagenet',
 'resnet18_imagenet',
 'inception_resnet_v2_imagenet',
 'resnext50_32x4d_imagenet',
 'ctranspath',
 'wide_resnet50_2_imagenet',
 'resnet50_v2_imagenet',
 'alexnet_imagenet']

In [7]:



# features = sf.DatasetFeatures(
#     'densenet161-8d451a50.pth',
#     dataset=dataset
#     layers='sepconv3_bn'
# )


# retccl = build_feature_extractor(
#     'densenet_imagenet',
#     tile_px=299
# )

In [9]:
# Load a project and dataset.
P = sf.Project(
    root='sf2_proj',
    annotations='annotations.csv',
    create=True
)
P.add_source(
  name="my_datasets",
  slides="datasets/brca-psj-path/ndpi/",
  tiles='datasets/brca-psj-path/contest-phase-2/clam-preprocessing-train/patches/'
)

[17:57:35] INFO     Creating project at sf2_proj...

           INFO     Saved dataset source my_datasets to sf2_proj/datasets.json

In [10]:
# full_dataset = P.dataset(tile_px=299, tile_um=302)

# # Calculate features for this dataset.
# features = sf.DatasetFeatures(retccl, dataset)
# features.to_torch('bag_dir')

In [11]:
from slideflow.mil import mil_config

# config = mil_config('TransMIL', lr=1e-3)
# config = mil_config(..., trainer='clam')

#model trainer configuration
config = mil_config(model='transmil',trainer='fastai',batch_size=8, lr=1e-3)


In [12]:
!{sys.executable} -m pip install fastai

Looking in indexes: https://devpi.svc.ngsci.local/root/pypi/+simple/
You should consider upgrading via the '/opt/venv/default/bin/python -m pip install --upgrade pip' command.


In [11]:
!{sys.executable} -m pip install nystrom_attention

Looking in indexes: https://devpi.svc.ngsci.local/root/pypi/+simple/
You should consider upgrading via the '/opt/venv/default/bin/python -m pip install --upgrade pip' command.


In [12]:
P = sf.load_project('sf_proj')

In [13]:
torch.cuda.empty_cache()
full_dataset = P.dataset(tile_px=256, tile_um=256)
# Split the dataset using three-fold, site-preserved cross-validation
# splits = full_dataset.kfold_split(
#     k=3,
#     labels='stage',
#     preserved_site=False, 
#     from_wsi=True
# )

# # Train on each cross-fold
# for train, val in splits:
#     P.train_mil(
#         config=config,
#         outcomes='stage',
#         train_dataset=train,
#         val_dataset=val,
#         bags='datasets/brca-psj-path/contest-phase-2/clam-preprocessing-train/resnet50-features/pt_files/'
#     )

#split data into train and validation set
train_dataset = full_dataset.filter({"dataset": ["train"]})
val_dataset = full_dataset.filter({"dataset": ["eval"]})

#run trainer
P.train_mil(
    config=config,
    outcomes='stage',
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    from_wsi=True,
    starting_epoch=2,
    multi_gpu=True,
    save_model=True,
    save_checkpoint=True,
    validation_batch_size= 8,
    bags='datasets/brca-psj-path/contest-phase-2/clam-preprocessing-train/resnet50-features/pt_files'
    )

[17:57:45] INFO     Column 'slide' missing in annotations.

           INFO     Attempting to associate patients with slides...

[17:58:41] INFO     Done; associated slides with 10846 annotations.

           WARNING  tfrecords path not set for source my_datasets

           INFO     Training FastAI MIL model with config:

...

           INFO     Training model TransMIL, loss=CrossEntropyLoss

epoch,train_loss,valid_loss,roc_auc_score,time
0,0.176426,0.201070,0.564571,2:36:02
1,0.168276,0.185178,0.573556,2:47:13
2,0.175078,0.186332,0.548260,2:42:07
3,0.180708,0.186939,0.560674,2:39:52
4,0.186897,0.184211,0.572122,2:41:07
5,0.168908,0.197800,0.538804,2:40:17
6,0.167140,0.204255,0.544435,2:39:18
7,0.170284,0.200467,0.587654,2:33:44
8,0.165664,0.206838,0.594278,2:36:42
9,0.168115,0.179980,0.589516,2:33:28


Better model found at epoch 0 with valid_loss value: 0.20106999576091766.
Better model found at epoch 1 with valid_loss value: 0.18517804145812988.
Better model found at epoch 4 with valid_loss value: 0.1842106282711029.
Better model found at epoch 9 with valid_loss value: 0.17997997999191284.


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

In [15]:
torch.cuda.empty_cache()